In [29]:
import census
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [30]:
d = {}

for i in range(2009, 2019):
    d[i] = pd.read_csv("./Census"+str(i)+".csv")

#pip install census
#pip insrall us

import pandas as pd
from census import Census


YOUR_API_KEY = "205750d5f1d88da194e1e17c8cf525ac14452cfe"
CENSUS_FILE_NAME = 'mycensus'
variables = ['NAME', 'B01001_001E',  'B19013_001E', 'B25077_001E', 
             'B03002_003E', 'B03002_004E',  'B02001_004E', 'B03002_006E',  
             'B03002_007E', 'B03002_008E',  'B03002_009E', 'B03002_012E', 
            'B15002_032E', 'B15002_015E']#, 'B01001_010E', 'B01001_011E']

#for i in range(27, 35):
#    variables.append("B01001_0"+str(i)+"E")
    
#for i in range(3, 10):
#    variables.append("B01001_00"+str(i)+"E")

years = [2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018] 


c = Census(YOUR_API_KEY)


nyc_met_area = [
    {"state_code":"34", "county_code": "003", "county_name": "Bergen, NJ"},
    {"state_code":"34", "county_code": "013", "county_name": "Essex, NJ"},
    {"state_code":"34", "county_code": "017", "county_name": "Hudson, NJ"},
    {"state_code":"34", "county_code": "019", "county_name": "Hunterdon, NJ"},
    {"state_code":"34", "county_code": "023", "county_name": "Middlesex, NJ"},
    {"state_code":"34", "county_code": "025", "county_name": "Monmouth, NJ"},
    {"state_code":"34", "county_code": "027", "county_name": "Morris, NJ"},
    {"state_code":"34", "county_code": "029", "county_name": "Ocean, NJ"},
    {"state_code":"34", "county_code": "031", "county_name": "Passaic, NJ"},
    {"state_code":"34", "county_code": "035", "county_name": "Somerset, NJ"},
    {"state_code":"34", "county_code": "037", "county_name": "Sussex, NJ"},
    {"state_code":"34", "county_code": "039", "county_name": "Union, NJ"},
    {"state_code":"36", "county_code": "005", "county_name": "Bronx, NY"},
    {"state_code":"36", "county_code": "027", "county_name": "Dutchess, NY"},
    {"state_code":"36", "county_code": "047", "county_name": "Kings, NY"},
    {"state_code":"36", "county_code": "059", "county_name": "Nassau, NY"},
    {"state_code":"36", "county_code": "061", "county_name": "New York, NY"},
    {"state_code":"36", "county_code": "071", "county_name": "Orange, NY"},
    {"state_code":"36", "county_code": "079", "county_name": "Putnam, NY"},
    {"state_code":"36", "county_code": "081", "county_name": "Queens, NY"},
    {"state_code":"36", "county_code": "085", "county_name": "Richmond, NY"},
    {"state_code":"36", "county_code": "087", "county_name": "Rockland, NY"},
    {"state_code":"36", "county_code": "103", "county_name": "Suffolk, NY"},
    {"state_code":"36", "county_code": "119", "county_name": "Westchester, NY"},
    {"state_code":"42", "county_code": "103", "county_name": "Pike, PA"}  
]

dfCounties = pd.DataFrame(nyc_met_area)



def get_acs_data(c, variables, state_code, county_code, year):
    results = c.acs5.state_county_tract(
        variables,
        state_code,
        county_code, 
        Census.ALL,
        year = year
    )
    return results
    

   
for year in years:
    print('Year: {}'.format(year))

    census_data = []
    for county in nyc_met_area:
        print('      ' + county["county_name"])
        census_data +=  get_acs_data(c,
                                    variables,
                                    county["state_code"], 
                                    county["county_code"],
                                    year)
    
    df = pd.DataFrame(census_data)
    
    #create geoid columns
    df['geoid'] = df['state'] + df['county'] + df['tract']
    df['year'] = year
    
    #move it to the begining of the dataframe
    col = df.pop("year")
    df.insert(0, col.name, col)

    col = df.pop("geoid")
    df.insert(0, col.name, col)
    
    df.to_csv('{}_{}.csv'.format(CENSUS_FILE_NAME, str(year)+"_new"), index = False)


In [31]:
d2 = {}

for i in range(2009, 2019):
    d2[i] = pd.read_csv("./mycensus_"+str(i)+"_new"+".csv")

In [32]:
d2[2010].columns

Index(['geoid', 'year', 'NAME', 'B01001_001E', 'B19013_001E', 'B25077_001E',
       'B03002_003E', 'B03002_004E', 'B02001_004E', 'B03002_006E',
       'B03002_007E', 'B03002_008E', 'B03002_009E', 'B03002_012E',
       'B15002_032E', 'B15002_015E', 'state', 'county', 'tract'],
      dtype='object')

In [33]:
# to substract from total population to get ppl older than 25

#cols_substr = ['B01001_010E', 'B01001_011E',
#       'B01001_027E', 'B01001_028E', 'B01001_029E', 'B01001_030E',
#       'B01001_031E', 'B01001_032E', 'B01001_033E', 'B01001_034E',
#       'B01001_003E', 'B01001_004E', 'B01001_005E', 'B01001_006E',
#       'B01001_007E', 'B01001_008E', 'B01001_009E'] 

for i in range(2009, 2019):
    #d2[i]['25_and_older'] = d2[i].apply(lambda row: row['B01001_001E'] - sum([row[el] for el in cols_substr]), axis = 1)
    d2[i]['25_and_bachelors'] = d2[i].apply(lambda row: row['B15002_032E'] + row['B15002_015E'], axis = 1)
    d2[i]['percentage_of_bachelors'] = d2[i].apply(lambda row: row['25_and_bachelors']/row['B01001_001E'] if row['B01001_001E'] > 0 else 0, axis = 1)
    

Baseline: deciding which tracts gentrify between 2009 and 2018

In [34]:
# inflation from 2009 to 2018 was 1.17
inf_rate = 1.17#1.146

In [35]:
df_merged = d2[2018].merge(d2[2009], on = 'geoid', how = 'left')

In [36]:
# df_merged = df_merged[(df_merged['B25077_001E_y'] > 0) & (df_merged['B25077_001E_x'] > 0)& (df_merged['B19013_001E_y'] > 0) & (df_merged['B19013_001E_x'] > 0) ]

In [37]:
#df_merged = df_merged[(df_merged['B25077_001E_x'] > 0) & (df_merged['B25077_001E_y'] > 0) & (df_merged['B19013_001E_y'] > 0)]

In [38]:
df_merged['B19013_001E_x'] = df_merged['B19013_001E_x'].apply(lambda x: np.nan if x <= 0 else x)
df_merged['B19013_001E_y'] = df_merged['B19013_001E_y'].apply(lambda x: np.nan if x <= 0 else x)
df_merged['B25077_001E_x'] = df_merged['B25077_001E_x'].apply(lambda x: np.nan if x <= 0 else x)
df_merged['B25077_001E_y'] = df_merged['B25077_001E_y'].apply(lambda x: np.nan if x <= 0 else x)

In [39]:
#df_merged[(df_merged['B25077_001E_x'] >= 0) & (df_merged['B25077_001E_y'] >= 0) & (df_merged['B19013_001E_y'] >= 0)]

In [40]:
#df_merged = df_merged[(df_merged['B25077_001E_x'] > 0) & (df_merged['B25077_001E_y'] > 0) & (df_merged['B19013_001E_y'] > 0)]

In [41]:
#df_merged['B25077_001E_y'] = df_merged['B25077_001E_y'].replace(0, pd.np.nan)

In [42]:
df_merged['ed_chng'] = df_merged['percentage_of_bachelors_x'] - df_merged['percentage_of_bachelors_y']

In [43]:
median_home_start_bottom_40th = df_merged['B25077_001E_y'].quantile(0.4)

median_income_start_bottom_40th = df_merged['B19013_001E_y'].quantile(0.4)

df_merged['home_pct_change'] = (
    (
        df_merged['B25077_001E_x'] - 
        (df_merged['B25077_001E_y'] * inf_rate)  
    ) / 
    df_merged['B25077_001E_x'] # current median home value 
)

median_home_change_top3rd = df_merged[df_merged['home_pct_change'] !=None ]['home_pct_change'].quantile(0.66)

In [44]:
educational_attainment_threshold = df_merged['ed_chng'].quantile(2.0/3)
print(df_merged['ed_chng'].min())
print(df_merged['ed_chng'].max())
print(educational_attainment_threshold)

-1.0
0.469626888683318
0.03492776912207123


We only consider homes with all values present (i.e. no -666666666) for gentrification, as in https://www.governing.com/gov-data/gentrification-report-methodology.html

In [45]:
df_merged['has>500_start'] = df_merged['B01001_001E_y'] >= 500
df_merged['has>500_end'] = df_merged['B01001_001E_x'] >= 500
df_merged['median40start'] = (df_merged['B25077_001E_y'] < median_home_start_bottom_40th) & (df_merged['B25077_001E_y'] > 0) & (df_merged['B25077_001E_x'] > 0)  
# not -66666666 or NaN, i.e. valid value
df_merged['median40income'] = (df_merged['B19013_001E_y'] < median_income_start_bottom_40th) & (df_merged['B19013_001E_y'] > 0) & (df_merged['B19013_001E_x'] > 0)
df_merged['ed_chng_upper_third'] = df_merged['ed_chng'] > educational_attainment_threshold
df_merged['median_home_higher'] = df_merged['home_pct_change'] > 0
df_merged['median_home_topthird'] = df_merged['home_pct_change'] > median_home_change_top3rd

In [46]:
df_merged['eligible'] = (
    (df_merged['has>500_start']) &
    (df_merged['has>500_end']) &
    (df_merged['median40start']) &
    (df_merged['median40income'])
)

df_merged['gentrified'] = (
    (df_merged['eligible']) &
    (df_merged['ed_chng_upper_third']) &
    (df_merged['median_home_higher']) &
    (df_merged['median_home_topthird'])
)

In [47]:
data_gentrified_09_18 = df_merged[df_merged['gentrified']]
data_gentrified_09_18

,geoid,year_x,NAME_x,B01001_001E_x,B19013_001E_x,B25077_001E_x,B03002_003E_x,B03002_004E_x,B02001_004E_x,B03002_006E_x,...,home_pct_change,has>500_start,has>500_end,median40start,median40income,ed_chng_upper_third,median_home_higher,median_home_topthird,eligible,gentrified
231,34013008700,2018,"Census Tract 87, Essex County, New Jersey",3399.0,44302.0,286400.0,122.0,912.0,40.0,23.0,...,0.120866,True,True,True,True,True,True,True,True,True
247,34013006200,2018,"Census Tract 62, Essex County, New Jersey",1583.0,27564.0,87100.0,37.0,1036.0,15.0,17.0,...,0.872388,True,True,True,True,True,True,True,True,True
391,34017001202,2018,"Census Tract 12.02, Hudson County, New Jersey",1357.0,44886.0,381300.0,131.0,232.0,12.0,365.0,...,0.107081,True,True,True,True,True,True,True,True,True
412,34017004700,2018,"Census Tract 47, Hudson County, New Jersey",2777.0,61842.0,437300.0,668.0,605.0,0.0,146.0,...,0.059024,True,True,True,True,True,True,True,True,True
427,34017001000,2018,"Census Tract 10, Hudson County, New Jersey",1964.0,67826.0,362500.0,494.0,60.0,0.0,860.0,...,0.092080,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3898,36081054700,2018,"Census Tract 547, Queens County, New York",5221.0,62394.0,628500.0,901.0,177.0,0.0,596.0,...,0.320897,True,True,True,True,True,True,True,True,True
3901,36081055900,2018,"Census Tract 559, Queens County, New York",1113.0,64125.0,598200.0,123.0,50.0,0.0,88.0,...,0.242101,True,True,True,True,True,True,True,True,True
3928,36081064101,2018,"Census Tract 641.01, Queens County, New York",2239.0,72576.0,226100.0,1561.0,15.0,0.0,255.0,...,0.052512,True,True,True,True,True,True,True,True,True
4568,36119002203,2018,"Census Tract 22.03, Westchester County, New York",1744.0,62712.0,178400.0,1000.0,66.0,0.0,190.0,...,0.068722,True,True,True,True,True,True,True,True,True


In [48]:
data_gentrified_09_18.to_csv('./data_gentrified_09_18.csv')

In [49]:
data_eligible_not_g_09_18 = df_merged[(df_merged['eligible']) & (df_merged['gentrified'] == False)]

In [50]:
data_eligible_not_g_09_18.to_csv('./data_eligible_not_09_18.csv')
data_eligible_not_g_09_18

,geoid,year_x,NAME_x,B01001_001E_x,B19013_001E_x,B25077_001E_x,B03002_003E_x,B03002_004E_x,B02001_004E_x,B03002_006E_x,...,home_pct_change,has>500_start,has>500_end,median40start,median40income,ed_chng_upper_third,median_home_higher,median_home_topthird,eligible,gentrified
50,34003021200,2018,"Census Tract 212, Bergen County, New Jersey",5711.0,65417.0,335400.0,3811.0,69.0,0.0,158.0,...,-0.391512,True,True,True,True,False,False,False,True,False
51,34003029100,2018,"Census Tract 291, Bergen County, New Jersey",4440.0,52094.0,300000.0,1436.0,133.0,0.0,1761.0,...,-0.278420,True,True,True,True,False,False,False,True,False
63,34003018100,2018,"Census Tract 181, Bergen County, New Jersey",7425.0,55958.0,340600.0,1897.0,90.0,57.0,242.0,...,-0.361679,True,True,True,True,False,False,False,True,False
64,34003021300,2018,"Census Tract 213, Bergen County, New Jersey",4443.0,60255.0,320700.0,2941.0,70.0,0.0,121.0,...,-0.293676,True,True,True,True,False,False,False,True,False
66,34003023100,2018,"Census Tract 231, Bergen County, New Jersey",3234.0,58693.0,188900.0,720.0,679.0,8.0,313.0,...,-0.790614,True,True,True,True,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4654,36119014100,2018,"Census Tract 141, Westchester County, New York",6000.0,49002.0,310800.0,2367.0,1328.0,25.0,167.0,...,-0.397375,True,True,True,True,False,False,False,True,False
4657,36119001505,2018,"Census Tract 15.05, Westchester County, New York",2071.0,68150.0,122000.0,764.0,606.0,0.0,29.0,...,-0.307139,True,True,True,True,True,False,False,True,False
4659,36119003200,2018,"Census Tract 32, Westchester County, New York",2387.0,48917.0,318400.0,16.0,1921.0,68.0,0.0,...,-0.308901,True,True,True,True,False,False,False,True,False
4667,36119003600,2018,"Census Tract 36, Westchester County, New York",3559.0,35147.0,406900.0,559.0,1497.0,22.0,128.0,...,-0.157061,True,True,True,True,True,False,False,True,False
